# 데이터 불러오기

In [0]:
df_flight = spark.table("`1team-postgresql-connection_catalog`.gold.df_flight")



In [0]:
df_api = spark.table("`1team-postgresql-connection_catalog`.gold.df_api")

In [0]:
df_flight.show(5)
df_api.show(5)


#모델 분석

## 희망 모델
-선형회귀
-랜덤포레스트
-xgboost
-ligthgbm
-fcnn(딥러닝)
-릿지회귀
-catboost

In [0]:
# Spark DataFrame → Pandas
df = df_flight.toPandas()

# ✅ 공통 결측 제거
df = df.dropna()

# ✅ 범주형 인코딩 (원핫)
df_encoded = pd.get_dummies(df, columns=["congestion_level"], drop_first=True)

# ✅ 입력(X), 출력(y)
X_base = df_encoded.drop(columns=["delayed_time"])
y = df_encoded["delayed_time"]


### 선형회귀

In [0]:
# 모델 이름별 결과 저장을 위한 리스트 생성
model_results = []


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# 🔹 1. 입력 복사
X = X_base.copy()

# 🔹 2. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 🔹 3. 모델 학습
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# 🔹 4. 예측
y_pred = model_lr.predict(X_test)

# 🔹 5. 성능 평가
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("📊 선형 회귀 성능")
print(f"MAE  : {mae:.2f}분")
print(f"RMSE : {rmse:.2f}분")
print(f"R²   : {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'Linear Regression',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


### 랜덤포레스트

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 🔹 1. 입력 복사 (방금 만든 공통 X_base 활용)
X = X_base.copy()

# 🔹 2. 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 🔹 3. 모델 생성
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 🔹 4. 학습
rf_model.fit(X_train, y_train)

# 🔹 5. 예측
y_pred = rf_model.predict(X_test)

# 🔹 6. 성능 평가
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("🌲 랜덤포레스트 회귀 성능")
print(f"MAE  : {mae:.2f}분")
print(f"RMSE : {rmse:.2f}분")
print(f"R²   : {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'Random Forest',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


In [0]:
import matplotlib.pyplot as plt
import numpy as np
#피쳐중요도 확인
# 🔹 중요도 추출
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# 🔹 시각화
plt.figure(figsize=(10, 5))
plt.title("🎯 Variable Importance - Random Forest")
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=45, ha='right')
plt.tight_layout()
plt.show()


###xgboost

In [0]:
%pip install xgboost

In [0]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ⛳ 에러 해결을 위한 데이터 타입 정리
X = X_base.copy().apply(pd.to_numeric, errors='coerce')  # <-- 핵심
y = y.copy()

# 결측값 제거 (NaN 발생했을 수도 있음)
combined = pd.concat([X, y], axis=1).dropna()
X = combined.drop(columns=["delayed_time"])
y = combined["delayed_time"]

# Train/Test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 생성 및 학습
xgb_model = XGBRegressor(n_estimators=100, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("📈 XGBoost 회귀 성능")
print(f"MAE  : {mae:.2f}분")
print(f"RMSE : {rmse:.2f}분")
print(f"R²   : {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'XGBoost',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


### lightgbm

In [0]:
%pip install lightgbm


In [0]:
# 1. 라이브러리 로드 및 패키지 설치
# %pip install lightgbm

import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# 2. 데이터 준비 및 전처리 (df_flightDataFrame이 이미 Spark/Pandas로 있는 상태라고 가정)
# (필요하면 Databricks에서: df = df_flight.toPandas())
# 그리고 아래와 같이 공통파이프라인 적용:
# 범주형 인코딩(원핫), 결측치 제거, 타입 변환

df = df_flight.toPandas().dropna()
df_encoded = pd.get_dummies(df, columns=["congestion_level"], drop_first=True)
X_base = df_encoded.drop(columns=["delayed_time"])
y = df_encoded["delayed_time"]

# 타입 통일
X = X_base.copy().apply(pd.to_numeric, errors='coerce')
y_temp = y.copy()

# 결측치 합동 처리
combined = pd.concat([X, y_temp], axis=1).dropna()
X = combined.drop(columns=["delayed_time"])
y = combined["delayed_time"]

# 3. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. LGBM 모델 생성, 학습
lgb_model = LGBMRegressor(
    n_estimators=100,
    max_depth=-1,     
    random_state=42
)
lgb_model.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = lgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("✨ LightGBM 회귀 성능")
print(f"MAE  : {mae:.2f}분")
print(f"RMSE : {rmse:.2f}분")
print(f"R²   : {r2:.3f}")




In [0]:
model_results.append({
    '모델': 'LightGBM',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


In [0]:
# 6. 변수 중요도 시각화 (선택이지만 인기 있음)
importances = lgb_model.feature_importances_
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 5))
plt.title("LightGBM Feature Importance")
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=45, ha='right')
plt.tight_layout()
plt.show()

### 딥러닝(fcnn, keras)

In [0]:
%pip install tensorflow

In [0]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow import keras

# 데이터 준비 (공통 파이프라인 적용)
df = df_flight.toPandas().dropna()
df_encoded = pd.get_dummies(df, columns=["congestion_level"], drop_first=True)
X_base = df_encoded.drop(columns=["delayed_time"])
y = df_encoded["delayed_time"]
X = X_base.copy().apply(pd.to_numeric, errors='coerce')
y_temp = y.copy()
combined = pd.concat([X, y_temp], axis=1).dropna()
X = combined.drop(columns=["delayed_time"])
y = combined["delayed_time"]

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y.values, test_size=0.2, random_state=42
)

# 딥러닝(Fully Connected Neural Network)
model = keras.models.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mae')
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=0)

# 예측 및 평가
y_pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"🤖 FCNN(딥러닝) 성능\nMAE: {mae:.2f}분 | RMSE: {rmse:.2f}분 | R^2: {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'FCNN (딥러닝)',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


### 릿지회귀

In [0]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 데이터 준비 (동일 프로세스)
df = df_flight.toPandas().dropna()
df_encoded = pd.get_dummies(df, columns=["congestion_level"], drop_first=True)
X_base = df_encoded.drop(columns=["delayed_time"])
y = df_encoded["delayed_time"]
X = X_base.copy().apply(pd.to_numeric, errors='coerce')
y_temp = y.copy()
combined = pd.concat([X, y_temp], axis=1).dropna()
X = combined.drop(columns=["delayed_time"])
y = combined["delayed_time"]

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 릿지 회귀 모델 생성 및 학습
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# 예측 및 평가
y_pred = ridge_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"🔵 Ridge 회귀 성능\nMAE: {mae:.2f}분 | RMSE: {rmse:.2f}분 | R^2: {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'Ridge Regression',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


### catboost regressor

In [0]:
%pip install catboost

In [0]:


from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 데이터 준비 (공통)
df = df_flight.toPandas().dropna()
df_encoded = pd.get_dummies(df, columns=["congestion_level"], drop_first=True)
X_base = df_encoded.drop(columns=["delayed_time"])
y = df_encoded["delayed_time"]
X = X_base.copy().apply(pd.to_numeric, errors='coerce')
y_temp = y.copy()
combined = pd.concat([X, y_temp], axis=1).dropna()
X = combined.drop(columns=["delayed_time"])
y = combined["delayed_time"]

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# CatBoost 모델 생성, 학습, 평가
cat_model = CatBoostRegressor(verbose=0, iterations=100, random_seed=42)
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"🐱 CatBoost 회귀 성능\nMAE: {mae:.2f}분 | RMSE: {rmse:.2f}분 | R^2: {r2:.3f}")


In [0]:
model_results.append({
    '모델': 'CatBoost',
    'MAE': mae,
    'RMSE': rmse,
    'R2': r2
})


### GBTregressor( boosted decision tree regression)

❌ Spark MLlib 사용하는 GBTRegressor, StringIndexer, VectorAssembler 등은
→ Py4JSecurityException 등의 문제로 Databricks 제한 환경에서 통제될 수 있습니다.

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import when, col

# Step 1: delayed_time 있는 데이터만 사용
df_clean = df_flight.dropna(subset=["delayed_time"])

# Step 2: 혼잡도(범주형) 인코딩 → 수치형 (0=여유, 1=보통, 2=혼잡)
df_encoded = df_clean.withColumn(
    "congestion_index",
    when(col("congestion_level") == "여유", 0)
    .when(col("congestion_level") == "보통", 1)
    .when(col("congestion_level") == "혼잡", 2)
    .otherwise(None)
)

# Step 3: features 벡터 생성 (Builder 패턴 사용!)
assembler = VectorAssembler() \
    .setInputCols([
        "hour_of_day",
        "passenger_forecast",
        "temperature",
        "wind_speed",
        "visibility",
        "is_holiday",
        "congestion_index"
    ]) \
    .setOutputCol("features")

# Step 4: GBT Regressor 정의 (Builder 생성 방식)
gbt = GBTRegressor() \
    .setLabelCol("delayed_time") \
    .setFeaturesCol("features") \
    .setMaxIter(100) \
    .setMaxDepth(5) \
    .setSeed(42)

# Step 5: 파이프라인 연결
pipeline = Pipeline(stages=[assembler, gbt])

# Step 6: Train/Test Split
train_data, test_data = df_encoded.randomSplit([0.8, 0.2], seed=42)

# Step 7: 학습
model = pipeline.fit(train_data)

# Step 8: 예측
predictions = model.transform(test_data)

# Step 9: 성능 평가
evaluator = RegressionEvaluator() \
    .setLabelCol("delayed_time") \
    .setPredictionCol("prediction") \
    .setMetricName("mae")
mae = evaluator.evaluate(predictions)

rmse = RegressionEvaluator(labelCol="delayed_time", predictionCol="prediction", metricName="rmse").evaluate(predictions)
r2 = RegressionEvaluator(labelCol="delayed_time", predictionCol="prediction", metricName="r2").evaluate(predictions)

# Step 10: 결과 출력
print("✅ ✅ GBTRegressor (Boosted Tree) 결과")
print(f"MAE  : {mae:.2f}분")
print(f"RMSE : {rmse:.2f}분")
print(f"R²   : {r2:.3f}")


### 성능비교

In [0]:
import pandas as pd

# 비교 표로 변환
results_df = pd.DataFrame(model_results)

# MAE 기준 정렬
results_df = results_df.sort_values(by='MAE', ascending=True).reset_index(drop=True)

# 결과 출력
print("📊 모델별 성능 비교표 (MAE 기준 정렬)")
display(results_df)


### 모델 성능 비교 요약

| 순위 | 모델             | MAE   | RMSE  | R²     | 해석 요약                                 |
|-----:|------------------|-------|-------|--------|--------------------------------------------|
| 1    | _FCNN (딥러닝)_   | 20.26 | 52.95 | -0.036 | MAE 최저 (_예측값 안정적_), R²는 낮음       |
| 2    | XGBoost          | 21.28 | 48.77 | 0.121  | R² 최고, MAE는 2위                           |
| 3    | LightGBM         | 21.63 | 48.93 | 0.115  | 성능 균형, XGBoost와 유사                   |
| 4    | CatBoost         | 21.65 | 49.00 | 0.112  | LightGBM과 거의 동일 성능                   |
| 5    | Random Forest    | 22.02 | 52.19 | -0.007 | MAE/R² 모두 하락                            |
| 6    | Ridge Regression | 23.45 | 51.85 | 0.006  | 전통회귀 기준선                             |
| 7    | Linear Regression| 23.45 | 51.85 | 0.006  | Ridge와 유사                                |
```



### 💡 모델 비교 요약 및 최종 선택

🔹 **MAE 기준 최상**
- _FCNN (딥러닝)_ → **20.26분**
  - 한 건당 예측 오류가 평균 20분으로 가장 작음
  - 예측값이 비교적 **안정적**임

🔹 **R² 기준 최상 (설명력)**
- **XGBoost** → **R² = 0.121**
  - 약 **12%의 분산 설명력**을 가짐
  - 절대 수치는 높지 않지만, **모델 중 가장 높음**

🔹 **모델 간 전반 비교**
- _XGBoost_, _LightGBM_, _CatBoost_는 거의 유사한 성능군
  - MAE: **21.5~21.6**, R²: **0.11~0.12**
- _Linear Regression_, _Ridge_는 기준선(Baseline) 역할로 적합
- _Random Forest_는 추가 튜닝(깊이 조정 등) 후 재검토 필요

---

### ✅ 최종 모델 선택 기준 및 판단

> MAE 기준으로는 **FCNN (딥러닝)**이 가장 낮은 예측 오차를 보였으나,  
> **설명력(R²)** 측면에서는 **XGBoost**가 가장 우수한 성능을 보였습니다.

> 예측 오류가 일정 수준 이상 줄어든 상황에서,  
> **모델의 해석 가능성과 설명력**이 중요한 특성으로 판단됩니다.

> 따라서 실제 서비스에 적용할 모델로는  
> `**XGBoost**`를 채택하는 것이 가장 합리적인 선택입니다.
```

## 👇 결과 미리보기 (Databricks에서 렌더링 시)

### 💡 모델 비교 요약 및 최종 선택

🔹 **MAE 기준 최상**
- _FCNN (딥러닝)_ → **20.26분**
  - 한 건당 예측 오류가 평균 20분으로 가장 작음
  - 예측값이 비교적 **안정적**임

🔹 **R² 기준 최상 (설명력)**
- **XGBoost** → **R² = 0.121**
  - 약 **12%의 분산 설명력**을 가짐  
  - 절대 수치는 높지 않지만, **모델 중 가장 높음**

🔹 **모델 간 전반 비교**
- _XGBoost_, _LightGBM_, _CatBoost_는 거의 유사한 성능군
  - MAE: **21.5~21.6**, R²: **0.11~0.12**
- _Linear Regression_, _Ridge_는 기준선(Baseline) 역할로 적합
- _Random Forest_는 추가 튜닝(깊이 조


| 모델                          | R²(Coefficient_of_Determination) | MAE(Mean_Absolute_Error) | Relative_Absolute_Error | Relative_Squared_Error | RMSE(Root_Mean_Squared_Error) |
|-------------------------------|----------------------------------|-------------------------|------------------------|------------------------|-------------------------------|
| Linear Regression (선형 회귀) | 0.0963                           | 22.33                   | 0.9619                 | 0.9037                 | 48.57                         |
| Decision Forest Regression    | 0.0721                           | 20.74                   | 0.8932                 | 0.9279                 | 49.21                         |
| Boosted Decision Tree         | 0.1962                           | 20.58                   | 0.8863                 | 0.8038                 | 45.80                         |
| Neural Network Regression     | -0.0001                          | 23.28                   | 1.0028                 | 1.0000                 | 51.08                         |
| Poisson Regression            | 0.0954                           | 22.32                   | 0.9613                 | 0.9046                 | 48.58                         |
```

ML 기준 모델



| 모델                             | R²(Coefficient_of_Determination) | MAE(Mean_Absolute_Error) | Relative_Absolute_Error | Relative_Squared_Error | RMSE(Root_Mean_Squared_Error) |
|-----------------------------------|----------------------------------|-------------------------|------------------------|-----------------------|-------------------------------|
| Linear Regression(선형 회귀)      | 0.7086703                        | 10.26241                | 0.4420651              | 0.2913297             | 27.57181                      |
| decision forest regression        | 0.9555479                        | 5.055495                | 0.2177713              | 0.0444521             | 10.77008                      |
| boosted decision tree regression  | 0.9601012                        | 5.951071                | 0.2563493              | 0.03989882            | 10.20359                      |
| neural network regression         | -0.0000096                       | 23.28129                | 1.002869               | 1.00001               | 51.08284                      |
| poisson regression                | -0.1730987                       | 8.083411                | 0.3482023              | 1.173099              | 48.58476                      |
```
ml 전체컬럼

#모델 선정 후 등록

In [0]:
import joblib
joblib.dump(xgb_model, "final_xgboost_model.pkl")


# 실시가 ㄴ데이터 추론

In [0]:
required_features = [
    'hour_of_day',
    'passenger_forecast',
    'temperature',
    'wind_speed',
    'visibility',
    'is_holiday',
    'congestion_level_여유',
    'congestion_level_혼잡'
]


In [0]:
# Step 1: 필요 없는 항목 제거
df_api = df_api.drop(columns=["year", "month", "day", "congestion_level"], errors="ignore")

# Step 2: 필요한 컬럼 없으면 생성 => 기본값 넣기
for col in ['is_holiday', 'congestion_level_여유', 'congestion_level_혼잡']:
    if col not in df_api.columns:
        df_api[col] = 0  # 예: 실제 예측 상황에서 default는 0으로

# Step 3: 컬럼 순서도 통일
df_api = df_api[required_features]

# Step 4: 숫자로 처리 + 결측 제거
df_api = df_api.apply(pd.to_numeric, errors='coerce').dropna()

# Step 5: 예측
y_pred = xgb_model.predict(df_api)


In [0]:
df_api["predicted_delayed_time"] = y_pred
display(df_api.head())  # Databricks 노트북에서 표 보기



In [0]:
from pyspark.sql import SparkSession

# pandas → spark로 다시 변환
df_api_spark_result = spark.createDataFrame(df_api)

# 저장 
df_api_spark_result.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "gold.df_api_pred") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()


In [0]:
import matplotlib.pyplot as plt

plt.hist(df_api["predicted_delayed_time"], bins=30)
plt.title("Predicted Delayed Time Histogram")
plt.xlabel("Predicted Delay (minutes)")
plt.ylabel("Count")
plt.show()


 5. (선택) 대시보드 연동, 자동 워크플로 스케줄
Databricks [Workflows]에서 노트북 실행 스케줄 설정 (10분/1시간 등 반복)

대시보드에 df_api_pred 테이블 연동, 실시간 예측값 모니터링

In [0]:
# 🔹 1. 필수 라이브러리
import joblib
import pandas as pd

# 🔹 2. 저장된 XGBoost 모델 로드
xgb_model = joblib.load("final_xgboost_model.pkl")

# 🔹 3. 실시간 데이터 (gold.df_api) PostgreSQL에서 로드
df_api_spark = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "gold.df_api") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_api = df_api_spark.toPandas()

# 🔹 4. 전처리 - feature 맞추기
required_features = [
    "hour_of_day",
    "passenger_forecast",
    "temperature",
    "wind_speed",
    "visibility",
    "is_holiday",
    "congestion_level_여유",
    "congestion_level_혼잡"
]

# ✅ 원핫 인코딩 (congestion_level → 더미 변수)
if "congestion_level" in df_api.columns:
    df_api = pd.get_dummies(df_api, columns=["congestion_level"], drop_first=True)

# ✅ 누락된 컬럼 생성 (기본값 0)
for col in required_features:
    if col not in df_api.columns:
        df_api[col] = 0

# ✅ 예측용 컬럼 순서 정렬 및 숫자 변환 + 결측 처리
X_pred = df_api[required_features]
X_pred = X_pred.apply(pd.to_numeric, errors="coerce").dropna()

# 🔹 5. 예측 실행
y_pred = xgb_model.predict(X_pred)

# 🔹 6. 결과 조합 (예측값 붙이기)
df_api.loc[X_pred.index, "predicted_delayed_time"] = y_pred

# 🔹 7. 결과 테이블 구성: year, month, day, hour_of_day, predicted_delayed_time
df_result = df_api[["year", "month", "day", "hour_of_day", "predicted_delayed_time"]]

# ✅ 8. Pandas → Spark 변환
df_result_spark = spark.createDataFrame(df_result)

# ✅ 9. PostgreSQL 의 gold.df_api_pred 테이블에 덮어쓰기 저장
df_result_spark.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "gold.df_api_pred") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

# 🔹 10. 확인용 출력
display(df_result_spark)


# 데이터 보강후 모델 재실행
 ## 바보마냥 도착기상(출발 및 도착 관련 중요 변수) 안넣고 모델링함 눈물 머금고 다시하는중 ㅠㅠ

In [0]:
spark.sql("USE CATALOG `1team-postgresql-connection_catalog`")
spark.sql("USE SCHEMA gold")


In [0]:
df_flight = spark.table("gold.df_flight")
df_flight.printSchema()



In [0]:
df_pd = df_flight.toPandas()
print(df_pd.shape)



In [0]:
import pandas as pd
import numpy as np

# 주요 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score


In [0]:
def time_to_min(t):
    """'HH:MM' 문자열 → 분(int). 결측시 None"""
    if pd.isnull(t):
        return None
    h, m = map(int, str(t).split(':'))
    return h * 60 + m


In [0]:
def compute_departure_delay(row):
    sched = row['scheduled_time_min']
    dep = row['departure_time_min']
    if pd.isnull(sched) or pd.isnull(dep):
        return None
    # ▶️ 자정 넘김(출발 < 예정) 케이스 보정
    if dep < sched:
        dep += 24 * 60
    return dep - sched


In [0]:
df_pd['scheduled_time_min'] = df_pd['scheduled_time'].apply(time_to_min)
df_pd['departure_time_min'] = df_pd['departure_time'].apply(time_to_min)
df_pd['departure_delay_min'] = df_pd.apply(compute_departure_delay, axis=1)


In [0]:
# full datetime 생성
import pandas as pd

def combine_datetime(row, time_col):
    if pd.isnull(row[time_col]):
        return pd.NaT
    date_str = f"{row['year']:04d}-{row['month']:02d}-{row['day']:02d}"
    dt_str = f"{date_str} {row[time_col]}"
    return pd.to_datetime(dt_str, format="%Y-%m-%d %H:%M")

df_pd['scheduled_dt'] = df_pd.apply(lambda r: combine_datetime(r, 'scheduled_time'), axis=1)
df_pd['departure_dt'] = df_pd.apply(lambda r: combine_datetime(r, 'departure_time'), axis=1)

# 지연(분) 계산
df_pd['departure_delay_min'] = (df_pd['departure_dt'] - df_pd['scheduled_dt']).dt.total_seconds() // 60


In [0]:
df_pd['departure_delay_min_adj'] = df_pd['departure_delay_min'].clip(lower=0)


In [0]:
df_api_fixed = spark.createDataFrame(df_pd)

df_api_fixed.repartition(4).write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "gold.df_flight") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .option("batchsize", 500) \
    .mode("overwrite") \
    .save()


#진짜 찐 최종으로 api불러오기 오류나지마라

In [0]:
spark.sql("SELECT current_catalog()").show()
spark.sql("SELECT current_schema()").show()


##도착기상

In [0]:
# 전체 경로 지정해서 불러오기
df_departure = spark.table("silver.api_silver_departure_weather")
# 또는 카탈로그, 스키마 직접 선택
spark.sql("USE SCHEMA silver")
df_departure = spark.table("api_silver_departure_weather")


In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# 1. Window 정의: 같은 key마다 timestamp 역순 정렬
key_cols = ["year", "month", "day", "scheduledatetime", "flightid"]  # 실무엔 필요한 조합으로!
window_spec = Window.partitionBy(key_cols).orderBy(F.col("timestamp").desc())

# 2. row_number 부여 → 각 파티션별 가장 최근(row_number==1)만 남김
df_departure_latest = (
    df_departure
    .withColumn("row_number", F.row_number().over(window_spec))
    .filter("row_number = 1")
    .drop("row_number")
)


In [0]:
from pyspark.sql.functions import col, substring

df_dep_feature1 = (
    df_departure_latest
    .select(
        "year",
        "month",
        "day",
        "airline",
        "flightid",
        "scheduledatetime",
        "airport",
        col("temp").cast("float").alias("temp"),
        col("wind").cast("float").alias("wind"),
        col("senstemp").cast("float").alias("senstemp"),
        col("himidity").cast("float").alias("himidity"),
        # 필요하다면 이런 최소한의 파생만 허용:
        substring("scheduledatetime", 1, 2).alias("scheduled_hour"),
    )
)


In [0]:
df_departure=df_dep_feature1

## 인천공항 날씨

In [0]:
# 1. 테이블 불러오기 (Spark SQL)
df_area = spark.table("silver.api_silver_area_weather_data")

# 2. 중복제거: observ_date, observ_time 기준으로 가장 최근(timestamp)값 1개만 남기기
from pyspark.sql.window import Window
import pyspark.sql.functions as F

window_spec = Window.partitionBy("observ_date", "observ_time").orderBy(F.col("timestamp").desc())
df_area_clean = (
    df_area.withColumn("row_number", F.row_number().over(window_spec))
           .filter(F.col("row_number") == 1)
           .drop("row_number")
)

In [0]:
df_dep_feature2 = df_area_clean.select(
    "observ_date",          # 조인키
    "observ_time",          # 조인키
    "temperature",          # 기온
    "mean_wind_speed",      # 평균 풍속
    "horizontal_visibility" # 가시거리
)


In [0]:
df_weather= df_dep_feature2

## ㅇ인구예측

In [0]:
import pyspark.sql.functions as F

# 1. 원본 데이터 불러오기
df_forecast = spark.table("gold.api_gold_departure_forecast")  # 실제 환경에 맞게 테이블명 변경

# 2. 시간대별(passengers_forecast 합계) 집계
df_forecast_grouped = (
    df_forecast
    .groupBy("year", "month", "day", "hour_of_day")
    .agg(
        F.sum(F.col("passengers_forecast").cast("int")).alias("passengers_forecast")
    )
    .withColumn(
        "congestion_level",
        F.when(F.col("passengers_forecast") < 2500, "여유")
         .when(F.col("passengers_forecast") < 5000, "보통")
         .otherwise("혼잡")
    )
    .orderBy("hour_of_day")
)

## 3. 결과 확인 (최대 24시간치 표시)
df_forecast_grouped.show(24)



In [0]:
df_forecast=df_forecast_grouped

In [0]:
from pyspark.sql.functions import substring

# 예시: scheduledatetime(예: '0850')에서 시간대 추출 -> '08'
df_departure = df_departure.withColumn(
    "hour_of_day",
    substring("scheduledatetime", 1, 2)
)


In [0]:
import datetime

# 오늘 날짜 가져오기 (로컬 시스템 시간 기준)
today = datetime.date.today()
today_year = today.year
today_month = today.month
today_day = today.day

df_departure_today = df_departure.filter(
    (df_departure.year == today_year) &
    (df_departure.month == today_month) &
    (df_departure.day == today_day)
)


In [0]:
from pyspark.sql.functions import substring, lpad, col

# df_weather: hour_of_day 파생
df_weather = df_weather.withColumn("hour_of_day", substring("observ_time", 1, 2))
df_weather = df_weather.withColumn("hour_of_day", lpad(col("hour_of_day"), 2, '0'))

# df_forecast: lpad 처리
df_forecast = df_forecast.withColumn("hour_of_day", lpad(col("hour_of_day"), 2, '0'))

# 기준 (df_departure_today)도 똑같이 맞춰야 함
df_departure_today = df_departure_today.withColumn("hour_of_day", lpad(col("hour_of_day"), 2, '0'))

# 조인
joined_df = (
    df_departure_today
        .join(df_weather, ["year", "month", "day", "hour_of_day"], "left")
        .join(df_forecast, ["year", "month", "day", "hour_of_day"], "left")
)


In [0]:
# 각 테이블 컬럼 리스트 출력
print("df_departure_today.columns:", df_departure_today.columns)



In [0]:
print("df_weather.columns:", df_weather.columns)


In [0]:
print("df_forecast.columns:", df_forecast.columns)

In [0]:
# df_weather에 조인키 생성 및 lpad 적용
from pyspark.sql.functions import substring, lpad, col

df_weather = (
    df_weather
    .withColumn("year", substring("observ_date", 1, 4))
    .withColumn("month", substring("observ_date", 5, 2))
    .withColumn("day", substring("observ_date", 7, 2))
    .withColumn("hour_of_day", substring("observ_time", 1, 2))
    .withColumn("hour_of_day", lpad(col("hour_of_day"), 2, '0'))
)

# 이제 모두 'year', 'month', 'day', 'hour_of_day' 기준 조인 OK!


In [0]:
joined_df = (
    df_departure_today
    .join(df_weather, ["year", "month", "day", "hour_of_day"], how='left')
    .join(df_forecast, ["year", "month", "day", "hour_of_day"], how='left')
)


In [0]:
joined_df.repartition(4).write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "gold.df_api") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .option("batchsize", 500) \
    .option("connectionTimeout", "60000") \
    .option("socketTimeout", "60000") \
    .mode("overwrite") \
    .save()


#분석시작

In [0]:
df_api=joined_df

In [0]:
# 정적 데이터 최종 피처셋 추출
df_flight_model = df_flight.select(
    "year",
    "month",
    "day",
    F.col("scheduled_hour").alias("hour_of_day"),
    "airline",
    F.col("flight_number").alias("flightid"),
    F.col("arrival_city").alias("airport"),
    "congestion_level",
    "temperature",
    F.col("wind_speed").alias("wind"),
    F.col("visibillity").alias("horizontal_visibility")
)


In [0]:
# 실시간 데이터 최종 피처셋 추출
df_api_model = df_api.select(
    "year",
    "month",
    "day",
    "hour_of_day",
    "airline",
    "flightid",
    "airport",
    "congestion_level",
    F.col("temp").alias("temperature"),
    "wind",
    F.col("horizontal_visibility")
)


In [0]:
X = df_flight_model.toPandas()
y = df_flight.select("departure_delay_min").toPandas().values.ravel()


In [0]:
!pip install xgboost


In [0]:
!pip install lightgbm


In [0]:
!pip install catboost


In [0]:
!pip install tensorflow


In [0]:
#!pip install scikit-learn pandas numpy


In [0]:
import pandas as pd
import numpy as np

# Sklearn & ML/DL 패키지
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 딥러닝(Fully Connected Neural Network)
import tensorflow as tf
from tensorflow.keras import layers, models


In [0]:
X = pd.get_dummies(X, columns=["airline", "airport", "congestion_level"])


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [0]:
models = {
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0),
    "RandomForest": RandomForestRegressor(),
    "Ridge": Ridge(),
    "Linear": LinearRegression()
}


In [0]:
# --- 기본 라이브러리 및 파이썬 ML 패키지 임포트 ---
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- 1. 정적 데이터 피처셋 추출 ---
df_flight_model = df_flight.select(
    "year",
    "month",
    "day",
    F.col("scheduled_hour").alias("hour_of_day"),
    "airline",
    F.col("flight_number").alias("flightid"),
    F.col("arrival_city").alias("airport"),
    "congestion_level",
    "temperature",
    F.col("wind_speed").alias("wind"),
    F.col("visibillity").alias("horizontal_visibility")
)

# --- 2. 피처/타깃 분리 및 데이터프레임 변환 ---
X = df_flight_model.toPandas()
y = df_flight.select("departure_delay_min").toPandas().values.ravel()

# --- 3. 식별자 컬럼 제거 및 인코딩/숫자형 변환 ---
# flightid(식별자)는 제외
X = X.drop("flightid", axis=1)
# 범주형 피처 원핫 인코딩
X = pd.get_dummies(X, columns=["airline", "airport", "congestion_level"])
# 수치형 컬럼 float 변환
for col in ["temperature", "wind", "horizontal_visibility"]:
    X[col] = X[col].astype(float)
# hour_of_day도 숫자 변환(문자일 경우)
if X["hour_of_day"].dtype == object:
    X["hour_of_day"] = X["hour_of_day"].astype(int)

# --- 4. 학습/테스트 데이터 분리 ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 5. 모델 정의 ---
models = {
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0),
    "RandomForest": RandomForestRegressor(),
    "Ridge": Ridge(),
    "Linear": LinearRegression()
}

# --- 6. 학습 및 평가 ---
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "MAE": mean_absolute_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred)
    }

# --- 7. 결과 출력 (표 형태) ---
print(pd.DataFrame(results).T)


In [0]:
results = {}

In [0]:


# 1. RandomForest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
results["RandomForest"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("RandomForest done")

In [0]:
# 2. LightGBM
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
results["LightGBM"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("LightGBM done")

In [0]:
# 3. XGBoost
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
results["XGBoost"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("XGBoost done")

In [0]:
# 4. CatBoost
cat = CatBoostRegressor(verbose=0)
cat.fit(X_train, y_train)
y_pred = cat.predict(X_test)
results["CatBoost"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("CatBoost done")

In [0]:
# 5. Ridge
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
results["Ridge"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("Ridge done")

In [0]:
# 6. Linear
linear = LinearRegression()
linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)
results["Linear"] = {
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2": r2_score(y_test, y_pred)
}
print("LinearRegression done")

In [0]:
import pandas as pd
df_results = pd.DataFrame(results).T
print(df_results)


| Model         |    RMSE   |     MAE    |    R2    |
|---------------|-----------|-----------|---------|
| RandomForest  | 44.27     | 19.07     | 0.2365  |
| LightGBM      | 43.55     | 19.90     | 0.2613  |
| XGBoost       | 43.62     | 19.72     | 0.2589  |
| CatBoost      | 43.49     | 19.58     | 0.2633  |
| Ridge         | 47.49     | 22.29     | 0.1214  |
| Linear        | 47.49     | 22.29     | 0.1214  |


### 모델별 성능결과 해석

- **CatBoost**가 RMSE(43.49), MAE(19.58)가 가장 낮으며, R²(0.263)도 가장 높음.  
  → 전체 모델 중 **가장 뛰어난 예측 성능**을 보임.
- **LightGBM, XGBoost, RandomForest**도 CatBoost와 매우 유사한 RMSE/MAE/R²를 보임.  
  → **트리 계열 모델 성능이 상향 평준화**된 모습
- **Ridge, Linear Regression**에서는 RMSE/MAE가 가장 높고 R²는 가장 낮음(0.12).  
  → **선형 계열 모델은 트리 계열에 비해 현저히 낮은 성능**을 보임.
- **전체적으로** 트리 계열(부스팅/랜덤포레스트)이 선형 계열보다  
  **항공편 출발 지연 예측에 더 적합**함을 확인할 수 있음.

**지표 의미**  
- RMSE(평균 제곱근 오차): 예측 오차의 표준적인 크기 (단위: 분, 작을수록 예측력 높음)
- MAE(평균 절대 오차): 실제 오차의 평균 절대값 (단위: 분, 작을수록 좋음)
- R²(결정계수): 1에 가까울수록 예측 적합도와 설명력이 우수

**실무 시사점**
- 트리 계열 모델은 출발지연 예측에서 가장 효과적인 접근법임
- 성능 차이가 미세하므로 최적화, 예측 시간, 실무 환경을 참고하여 모델 최종 선택 권장



# 모델 저장후 불러오기

In [0]:
from catboost import CatBoostRegressor

# 이미 CatBoost 모델을 학습한 경우
cat = CatBoostRegressor(verbose=0)
cat.fit(X_train, y_train)

# 모델을 로컬 PC에 저장 (예: 현재 작업 폴더에 'catboost_model.cbm' 파일 생성)
cat.save_model('catboost_model.cbm')


In [0]:
from catboost import CatBoostRegressor

cat_loaded = CatBoostRegressor()
cat_loaded.load_model('catboost_model.cbm')


In [0]:
joined_df.count()


In [0]:
total_row = joined_df.count()
batch_size = 5000
n_batch = (total_row // batch_size) + int(total_row % batch_size > 0)


In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# 1. joined_df에 row index 컬럼 추가
w = Window.orderBy(monotonically_increasing_id())
joined_df_idx = joined_df.withColumn("row_idx", row_number().over(w)-1)

pred_dfs = []

for i in range(n_batch):
    print(f"Processing batch {i+1}/{n_batch} ...")
    # 2. 배치 범위만 Spark DF로 추출
    start_idx = i * batch_size
    end_idx = min((i+1) * batch_size, total_row)
    batch_sdf = joined_df_idx.filter((F.col("row_idx") >= start_idx) & (F.col("row_idx") < end_idx)).drop("row_idx")
    # 3. Pandas로 변환
    X_real = batch_sdf.toPandas()
    # ---- 이하 기존 전처리, 예측 동일하게 ----
    if 'flightid' in X_real.columns:
        X_real = X_real.drop("flightid", axis=1)
    X_real = pd.get_dummies(X_real, columns=["airline", "airport", "congestion_level"])
    for col in ["temperature", "wind", "horizontal_visibility"]:
        X_real[col] = X_real[col].astype(float)
    if X_real["hour_of_day"].dtype == object:
        X_real["hour_of_day"] = X_real["hour_of_day"].astype(int)
    # feature_list = list(X.columns)  # 기존에 쓰던 변수 활용
    X_real = X_real.reindex(columns=feature_list, fill_value=0)
    # 예측
    y_pred_api = cat_loaded.predict(X_real)
    X_real["departure_delay_min_pred"] = y_pred_api
    pred_dfs.append(X_real[["departure_delay_min_pred"]])  # 필요한 컬럼만 쌓기


In [0]:
# 학습 시 전처리된 X 의 feature 리스트를 사용
# 예: X = 정적 데이터(df_flight_model) 전처리 후 사용했던 X
feature_list = list(X.columns)   # (X: df_flight_model → drop/인코딩/타입변환한 DataFrame)
# joined_df에서 누락된 피처가 있을 수 있으니 fill_value=0로 채우기
X_real = X_real.reindex(columns=feature_list, fill_value=0)


In [0]:
y_pred_api = cat_loaded.predict(X_real)
joined_df["departure_delay_min_pred"] = y_pred_api
